In [1]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import HourLocator, DateFormatter
import datetime
import numpy as np
import pandas as pd
import mplhep as hep

matplotlib.rcParams.update(matplotlib.rcParamsDefault)
plt.style.use(hep.style.CMS)


In [ ]:
oms_date_format = '%Y-%m-%d %H:%M:%S'
l1ad_dict = {
    'l1_phys' : {'key' : 'l1a_physics_rate', 'label' : 'L1 Physics Rate'},
    'looser'  : {'key' : 'l1_adt_80_rate', 'label' : 'L1AD Score > 5'},
    'loose'   : {'key' : 'l1_adt_400_rate', 'label' : 'L1AD Score > 25'},
    'tight'   : {'key' : 'l1_adt_4000_rate', 'label' : 'L1AD Score > 250'},
    'tighter' : {'key' : 'l1_adt_20000_rate', 'label' : 'L1AD Score > 1250'},
}

def get_color():
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][::-1]
    i = 0
    while True:
        yield colors[i]
        i = (i + 1) % len(colors)


timestamps = {
    # For Fill 8559
    '2023-04-09 08:35:27' : 'Run 365879 [circulating]', 
    '2023-04-09 08:52:17' : 'Run 365880 [collisions]', 
    '2023-04-09 09:13:53' : 'Run 365881 [collisions]', 
    '2023-04-09 09:18:52' : 'Run 365882 [collisions]', 
    '2023-04-09 11:08:05' : 'Stable Beam Start',
    '2023-04-09 13:25:31' : 'Run 365883 [collisions]', 
    '2023-04-09 13:30:53' : 'Run 365884 [collisions]', 
    '2023-04-09 13:50:24' : 'Run 365888 [collisions]', 
    '2023-04-09 13:57:22' : 'Run 365889 [collisions]', 
    '2023-04-09 16:59:40' : 'Stable Beam End',
    # For Fill 8550
    # '2023-04-07 11:42:32' :  '365825 [circulating]',
    # '2023-04-07 12:45:46' :  '365831 [circulating]',
    # '2023-04-07 12:51:39' :  '365833 [circulating]',
    # '2023-04-07 13:13:35' :  'Stable Beams Start',
    # '2023-04-07 13:16:35' :  '365834 [collisions]',
    # '2023-04-07 13:34:07' :  '365835 [collisions]',
    # '2023-04-07 13:50:13' :  '365836 [collisions]',
    # '2023-04-07 14:06:53' :  '365837 [collisions]',
    # '2023-04-07 14:24:05' :  '365838 [collisions]',
    # '2023-04-07 14:40:41' :  '365839 [collisions]',
    # '2023-04-07 14:57:55' :  '365840 [collisions]',
    # '2023-04-07 15:15:27' :  '365841 [collisions]',
    # '2023-04-07 15:35:22' :  '365842 [collisions]',
    # '2023-04-07 15:56:39' :  '365843 [collisions]',
    # '2023-04-07 18:23:20' :  '365844 [collisions]',
    # '2023-04-07 23:16:09' :  'Stable Beams End',
    # '2023-04-07 23:23:53' :  '365845 [cosmics]',
}

def plot_run(df, runnumber, l1ad_bits, label=None, year=2023, com=None, lumi=None, timestamps=None):
    run_df = rate_df.query(f'runnumber  == {float(runnumber)}')
    run_df.index.strftime('%H:%M')
    ax = run_df.plot(kind='line', use_index=True, y=[l1ad_dict[item]['key'] for item in l1ad_dict.keys() if item in l1ad_bits], figsize=(10, 10))
    ax.legend([l1ad_dict[item]['label'] for item in l1ad_dict.keys() if item in l1ad_bits])
    ax.set_ylabel('Rate [Hz]')
    ax.set_xlabel('Time')
    plt.yscale('log')

    if timestamps:
        yrange = ax.get_ylim()[1]-ax.get_ylim()[0]
        if 'log' in ax.get_yscale():
            y_values = np.logspace(np.log10(ax.get_ylim()[1])*.25, np.log10(ax.get_ylim()[1])*.88, num=len(timestamps))[::-1]
        else:
            y_values = np.linspace(ax.get_ylim()[1]*.25,ax.get_ylim()[1]*.88, num=len(timestamps))[::-1]
        colors = get_color()
        for i, (time, text) in enumerate((timestamps.items())):
            col = next(colors)
            plt.axvline(x=pd.Timestamp(time), linestyle='--', color=col)
            plt.text(x=pd.Timestamp(time), y=y_values[i], s=text, color=col, fontsize='xx-small')

    hep.cms.label(f'Run {runnumber}' if label is None else label, data=True, year=year, com=com,lumi=lumi)
    plt.show()

def plot_fill(df, runrange, l1ad_bits, label=None, year=2023, com=None, lumi=None, timestamps=None):
    # plt.figure(figsize=(10,2))
    run_df = rate_df.query(f'runnumber  >= {float(runrange[0])} & runnumber <= {float(runrange[1])}')
    ax = run_df.plot(kind='line', use_index=True, y=[l1ad_dict[item]['key'] for item in l1ad_dict.keys() if item in l1ad_bits], figsize=(20, 8))
    run_df.index.strftime('%H:%M')
    hour_locator = HourLocator(interval=1)
    minute_locator = MinuteLocator(interval=10) 
    ax.xaxis.set_major_locator(hour_locator)
    ax.xaxis.set_minor_locator(minute_locator)
    hour_formatter = DateFormatter('%H:%M') # format as hours and minutes
    ax.xaxis.set_major_formatter(hour_formatter)
    ax.legend([l1ad_dict[item]['label'] for item in l1ad_dict.keys() if item in l1ad_bits])
    ax.set_ylabel('Rate [Hz]')
    ax.set_xlabel('Time')
    plt.yscale('log')

    if timestamps:
        yrange = ax.get_ylim()[1]-ax.get_ylim()[0]
        if 'log' in ax.get_yscale():
            y_values = np.logspace(np.log10(ax.get_ylim()[1])*.25, np.log10(ax.get_ylim()[1])*.88, num=len(timestamps))[::-1]
        else:
            y_values = np.linspace(ax.get_ylim()[1]*.25,ax.get_ylim()[1]*.88, num=len(timestamps))[::-1]
        colors = get_color()
        for i, (time, text) in enumerate((timestamps.items())):
            col = next(colors)
            plt.axvline(x=pd.Timestamp(time), linestyle='--', color=col)
            plt.text(x=pd.Timestamp(time), y=y_values[i], s=text, color=col, fontsize='xx-small')

    hep.cms.label(f'Fill' if label is None else label, data=True, year=year, com=com,lumi=lumi)
    plt.show()

In [ ]:
rate_df = pd.read_pickle('L1AD_Monitoring.pkl')
plot_run(rate_df, 365882, ['l1_phys','loose'], lumi=0.000001, com=.45)
# plot_run(rate_df, 365833, ['l1_phys','loose'], lumi=0.000002, com=.45)

# Fill 8559
# plot_fill(rate_df, [365879,365889], ['l1_phys', 'looser', 'loose', 'tight', 'tighter'], label='Fill 8559', lumi=0.000001, com=.45, timestamps=timestamps)

# Fill 8550
# plot_fill(rate_df, [365824,365845], ['l1_phys', 'looser', 'loose', 'tight', 'tighter'], label='Fill 8550', lumi=0.000002, com=.45, timestamps=timestamps)